Notebook này là source code dùng để thu thập tất cả link của các loài động vật trên trang https://animalia.bio/

In [1]:
#Cài đặt selenium, chrome, chromedriver
#!pip install selenium
#!apt-get update 
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#Cài đặt beautifulsoup để crawl dữ liệu, vì beautifulsoup sẽ nhanh hơn selenium
#!pip install beautifulsoup4

In [2]:
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [3]:
#Khởi tạo driver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


In [5]:
# url của trang cần lấy dữ liệu
urls = {   
        'https://animalia.bio/mammals': 151 ,
        'https://animalia.bio/reptiles': 107, 
        'https://animalia.bio/amphibia' : 82,
        'https://animalia.bio/mollusk': 117,
        'https://animalia.bio/birds' : 247
}


In [6]:
#Hàm lấy dữ liệu của 1 trang rồi lưu nó xuống file html
def get_url(url, num_pages):
    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    driver.get(url)
    print(driver.current_url)
    page = 1
    while page < num_pages:
      try:
        driver.find_element("xpath", '/html/body/div[3]/div[2]/div/div[5]/div[3]/div[1]/span').click()
        page += 1
        print(page, num_pages)
      except:
        pass
    animals_class = url[21:]
    name = animals_class + '.html'
    with open(name, "w") as f:
        f.write(driver.page_source)     
    driver.close()
    

Cơ chế của website này là click vào button "LOAD MORE" thì sẽ cập nhật thêm dữ liệu của trang mới vào và dữ liệu trang cũ vẫn còn đó. Ví dụ trang 1 là dữ liệu của động vật có số thứ tự từ thứ 1-42 thì sau khi click vào thì dữ liệu trang 1 sẽ thành 1-84. Như vậy, ta sẽ click vào button "LOAD MORE" cho đến khi load hết tất cả dữ liệu . Mặc dù khi bấm vào button "LOAD MORE" thì url của trang có đổi nhưng nhóm vẫn không hiểu vì sao nếu nhập tay url vào thì nó tự reset về trang 1. Vì vậy nhóm phải dùng selenium để click vào button cho tới trang cuối cùng.

Test ở link :https://animalia.bio/mammals?page=1

In [ ]:
#Chạy tất cả các link, mỗi link xuất ra file csv
for url, num_pages in urls.items():
    get_url(url, num_pages)
    animals_class = url[21:]
    name = animals_class + '.html'
    #Dùng beautifulsoup để tìm tất cả các thẻ a
    with open(name) as fp:
        soup = BeautifulSoup(fp, "html.parser")
    animals_urls = soup.find_all("a", {"class": "animals-invert__item"}) #Tìm tất cả các link của thú rồi lưu vào biến

    animals_urls = [a['href'][:-11] for a in animals_urls] #Lấy link của từng thú, bỏ phần "?category=1"
    animals_urls_df = pd.DataFrame(data = animals_urls, columns = ["url"])
    animals_urls_df.to_csv(f'{animals_class}.csv') #Xuất kết quả ra file csv

Ở phần tìm các thẻ chứa link rồi lưu xuống file csv thì nhóm chọn BeautifulSoup vì nó đơn giản và nhanh hơn Selenium nhiều.

Bây giờ, kết quả đã lưu thành 5 file csv, đó là file "mammals.csv", "reptiles.csv", "amphibia.csv", "mollusk.csv" và "birds.csv". Cần tổng hợp lại thành 1 file duy nhất là "animals_urls.csv" để tiện hơn cho việc thu thập dữ liệu từng động vật.

In [23]:
list_csv = ["mammals.csv", "reptiles.csv", "amphibia.csv", "mollusk.csv", "birds.csv"]
animals = pd.Series(dtype = object)
for csv in list_csv:
    df = pd.read_csv(csv)
    animals = pd.concat([animals, df['url']], ignore_index=True)
df_animals = pd.DataFrame(animals, columns = ["url"])
df_animals.to_csv('animals_urls.csv')